# Semantic Segmentation Demo

This is a notebook for running the benchmark semantic segmentation network from the the [ADE20K MIT Scene Parsing Benchchmark](http://sceneparsing.csail.mit.edu/).

The code for this notebook is available here
https://github.com/CSAILVision/semantic-segmentation-pytorch/tree/master/notebooks

It can be run on Colab at this URL https://colab.research.google.com/github/CSAILVision/semantic-segmentation-pytorch/blob/master/notebooks/DemoSegmenter.ipynb

In [ ]:
import torch
torch.__version__

'2.5.1+cu121'

In [ ]:
an = [[-0.004716190509498119, -0.0015442944131791592, 0.0015981390606611967, -2.1806214135722257e-05, 7.439052569679917e-05, -1.0732343071140347e-05, 1.0], [0.02309301868081093, 0.0023342808708548546, 0.0506298653781414, 3.401018329896033e-05, -0.0019222927512601016, 0.0021084910258650775, 0.9999959468841552], [0.053202029317617416, 0.03648602962493896, 0.4865258038043976, -0.0004901473876088858, -0.0041520525701344005, -0.006891436409205198, 0.9999675154685974], [0.06836142390966415, -0.03956856951117516, 1.286145567893982, -0.0009973392589017747, 0.005531878676265478, -0.002320240251719952, 0.9999815821647645], [0.0203765332698822, -0.10057416558265686, 2.4121177196502686, -0.0005310514243319632, 0.01537159364670515, 0.001361840753816068, 0.9998807907104493], [0.045108165591955185, -0.14525069296360016, 3.55315899848938, 7.468684634659438e-05, 0.005704524926841259, 0.00235587265342474, 0.9999809861183167], [0.13488395512104034, -0.1963791251182556, 4.689173221588135, -0.0004860237531829625, -0.00040600186912342906, -0.004166984930634499, 0.9999911189079284], [0.1149468645453453, -0.2464183568954468, 5.962574481964111, -0.0004622219421435148, 0.00878873560577631, 0.0006996745360083878, 0.9999610781669616], [0.09449680894613266, -0.2945286333560944, 7.2549214363098145, -0.0008242461481131613, 0.01497135404497385, 0.002424252685159445, 0.9998847246170046], [0.1734085828065872, -0.35626524686813354, 8.757527351379393, 0.0006751691689714788, 0.0007519553182646632, -0.007941267453134062, 0.9999679923057555]]

In [ ]:
len(an)

10

### Environment Setup

First, download the code and pretrained models if we are on colab.

In [ ]:
%%bash
# Colab-specific setup
!(stat -t /usr/local/lib/*/dist-packages/google/colab > /dev/null 2>&1) && exit
pip install yacs 2>&1 >> install.log
git init 2>&1 >> install.log
git remote add origin https://github.com/CSAILVision/semantic-segmentation-pytorch.git 2>> install.log
git pull origin master 2>&1 >> install.log
DOWNLOAD_ONLY=1 ./demo_test.sh 2>> install.log

From https://github.com/CSAILVision/semantic-segmentation-pytorch
 * branch            master     -> FETCH_HEAD


In [ ]:
!git clone https://github.com/CSAILVision/semantic-segmentation-pytorch.git

Cloning into 'semantic-segmentation-pytorch'...
remote: Enumerating objects: 1170, done.
remote: Total 1170 (delta 0), reused 0 (delta 0), pack-reused 1170 (from 1)
Receiving objects: 100% (1170/1170), 5.04 MiB | 7.78 MiB/s, done.
Resolving deltas: 100% (707/707), done.


## Imports and utility functions

We need pytorch, numpy, and the code for the segmentation model.  And some utilities for visualizing the data.

In [ ]:
# System libs
import os, csv, torch, numpy, scipy.io, PIL.Image, torchvision.transforms
# Our libs
from mit_semseg.models import ModelBuilder, SegmentationModule
from mit_semseg.utils import colorEncode

# colors = scipy.io.loadmat('data/color150.mat')['colors']
# names = {}

# with open('data/object150_info.csv') as f:
#     reader = csv.reader(f)
#     next(reader)
#     for row in reader:
#         names[int(row[0])] = row[5].split(";")[0]

# def visualize_result(img, pred, index=None):
#     # filter prediction class if requested
#     if index is not None:
#         pred = pred.copy()
#         pred[pred != index] = -1
#         print(f'{names[index+1]}:')

#     # colorize prediction
#     pred_color = colorEncode(pred, colors).astype(numpy.uint8)

#     # aggregate images and save
#     im_vis = numpy.concatenate((img, pred_color), axis=1)
#     display(PIL.Image.fromarray(im_vis))

In [ ]:
ls

ckpt/          download_ADE20K.sh*  LICENSE      requirements.txt                teaser/
config/        eval_multipro.py     mit_semseg/  sample_data/                    test.py
data/          eval.py              notebooks/   semantic-segmentation-pytorch/  train.py
demo_test.sh*  install.log          README.md    setup.py


## Loading the segmentation model

Here we load a pretrained segmentation model.  Like any pytorch model, we can call it like a function, or examine the parameters in all the layers.

After loading, we put it on the GPU.  And since we are doing inference, not training, we put the model in eval mode.

In [ ]:
# Network Builders
net_encoder = ModelBuilder.build_encoder(
    arch='resnet50dilated',
    fc_dim=2048,
    weights='ckpt/ade20k-resnet50dilated-ppm_deepsup/encoder_epoch_20.pth')
net_decoder = ModelBuilder.build_decoder(
    arch='ppm_deepsup',
    fc_dim=2048,
    num_class=150,
    weights='ckpt/ade20k-resnet50dilated-ppm_deepsup/decoder_epoch_20.pth',
    use_softmax=True)

crit = torch.nn.NLLLoss(ignore_index=-1)
segmentation_module = SegmentationModule(net_encoder, net_decoder, crit)
segmentation_module.eval()
segmentation_module.cuda()

Loading weights for net_encoder


/content/mit_semseg/models/models.py:109: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(weights, map_location=lambda storage, loc: storage), strict=False)


Loading weights for net_decoder


/content/mit_semseg/models/models.py:156: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(weights, map_location=lambda storage, loc: storage), strict=False)


SegmentationModule(
  (encoder): ResnetDilated(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): SynchronizedBatchNorm2d(64, eps=1e-05, momentum=0.001, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): SynchronizedBatchNorm2d(64, eps=1e-05, momentum=0.001, affine=True, track_running_stats=True)
    (relu2): ReLU(inplace=True)
    (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn3): SynchronizedBatchNorm2d(128, eps=1e-05, momentum=0.001, affine=True, track_running_stats=True)
    (relu3): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): SynchronizedBatchNorm2d(64, eps=1

## Segmentation Mask

In [ ]:
# Load and normalize one image as a singleton tensor batch
pil_to_tensor = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(
        mean=[0.485, 0.456, 0.406], # These are RGB mean+std values
        std=[0.229, 0.224, 0.225])  # across a large photo dataset.
])



In [ ]:
pil_image = PIL.Image.open('ADE_val_00001519.jpg').convert('RGB')
img_original = numpy.array(pil_image)
img_data = pil_to_tensor(pil_image)
singleton_batch = {'img_data': img_data[None].cuda()}
output_size = img_data.shape[1:]

In [ ]:
import numpy as np
image_paths = ['ADE_val_00001519.jpg', 'ADE_val_00001519.jpg', 'ADE_val_00001519.jpg']
images = [PIL.Image.open(img_path).convert('RGB') for img_path in image_paths]
img_originals = [np.array(img) for img in images]

# Preprocess images and create batch
img_data = [pil_to_tensor(img)[None] for img in images]  # Normalize and add batch dimension
batch_data = torch.cat(img_data).cuda()  # Create a batch of images

# Run segmentation
segmentation_module.eval()
with torch.no_grad():
    scores = segmentation_module({'img_data': batch_data}, segSize=output_size)

# Get predicted segmentation masks
_, preds = torch.max(scores, dim=1)
pred_masks = preds.cpu().numpy()

## Run the Model

Finally we just pass the test image to the segmentation model.

The segmentation model is coded as a function that takes a dictionary as input, because it wants to know both the input batch image data as well as the desired output segmentation resolution.  We ask for full resolution output.

Then we use the previously-defined visualize_result function to render the segmentation map.

In [ ]:
# Run the segmentation at the highest resolution.
with torch.no_grad():
    scores = segmentation_module(singleton_batch, segSize=output_size)

# Get the predicted scores for each pixel
_, pred = torch.max(scores, dim=1)
pred = pred.cpu()[0].numpy()
# visualize_result(img_original, pred)

## Showing classes individually

To see which colors are which, here we visualize individual classes, one at a time.

In [ ]:
# # Top classes in answer
# predicted_classes = numpy.bincount(pred.flatten()).argsort()[::-1]
# for c in predicted_classes[:15]:
#     visualize_result(img_original, pred, c)

In [ ]:
pred.shape

(512, 768)

# Autoencoder

In [ ]:
import torch
import torch.nn as nn

class MaskAutoencoder(nn.Module):
    def __init__(self, height, width, k):
        super(MaskAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
        )
        self.flatten = nn.Flatten()
        # Calculating dimensions after downsampling
        def conv_output_size(input_size, kernel_size=3, stride=2, padding=1):
            return (input_size + 2*padding - kernel_size) // stride + 1

        h_after_conv1 = conv_output_size(height)
        w_after_conv1 = conv_output_size(width)
        h_after_conv2 = conv_output_size(h_after_conv1)
        w_after_conv2 = conv_output_size(w_after_conv1)

        self.encoded_dim = h_after_conv2 * w_after_conv2 * 64
        #self.encoded_dim = (height // 4) * (width // 4) * 64
        self.projection = nn.Linear(self.encoded_dim, k)

        # Decoder components
        self.unprojection = nn.Linear(k, self.encoded_dim)
        self.unflatten = nn.Unflatten(1, (64, h_after_conv2, w_after_conv2))
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 1, kernel_size=3, stride=2, padding=1, output_padding=1),
            #nn.PixelShuffle(1), # Not fix input and reconstructed size mismtach
            nn.Sigmoid()  # Sigmoid activation for pixel values in [0, 1]
        )

    def forward(self, x):
        # Encoding
        features = self.encoder(x)

        flattened = self.flatten(features)

        projected = self.projection(flattened)

        # Decoding
        unprojected = self.unprojection(projected)

        unflattened = self.unflatten(unprojected)

        reconstructed = self.decoder(unflattened)
        reconstructed = reconstructed[:, :, :x.shape[2], :x.shape[3]] # align output size with input size

        return projected, reconstructed

# Example reconstruction loss calculation
def reconstruction_loss(original, reconstructed):
    loss_fn = nn.MSELoss()  # Mean Squared Error for pixel-wise comparison
    return loss_fn(reconstructed, original)


In [ ]:
(270 // 4) * (480 // 4) * 64

514560

In [ ]:
import torch
import torch.nn as nn

height, width = pred.shape  # Example height and width
k = 648

simple_autoencoder = MaskAutoencoder(height, width, k)
simple_autoencoder = simple_autoencoder.cuda()

pred_tensor = torch.tensor(pred, dtype=torch.float32).unsqueeze(0).unsqueeze(0)  # Shape: (1, 1, height, width)
pred_tensor = pred_tensor.cuda()

# Forward pass through the simple autoencoder
encoded_mask, reconstructed_mask = simple_autoencoder(pred_tensor)

# Calculate reconstruction loss
reconstruction_loss = nn.MSELoss()
loss = reconstruction_loss(reconstructed_mask, pred_tensor)

print("Encoded Mask Shape:", encoded_mask.shape)
print("Reconstructed Mask Shape:", reconstructed_mask.shape)
print("Reconstruction Loss:", loss.item())


Encoded Mask Shape: torch.Size([1, 648])
Reconstructed Mask Shape: torch.Size([1, 1, 512, 768])
Reconstruction Loss: 833.5474853515625


In [ ]:

def process_image_and_get_loss(image_path, segmentation_module, autoencoder, normalization_params, device='cuda'):
    # Load and normalize the image
    pil_to_tensor = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(
        mean=[0.485, 0.456, 0.406], # These are RGB mean+std values
        std=[0.229, 0.224, 0.225])  # across a large photo dataset.
    ])

    pil_image = PIL.Image.open(image_path).convert('RGB')
    img_original = numpy.array(pil_image)
    img_data = pil_to_tensor(pil_image)
    singleton_batch = {'img_data': img_data[None].cuda()}
    output_size = img_data.shape[1:]


    # Run segmentation at the highest resolution
    with torch.no_grad():
        scores = segmentation_module(singleton_batch, segSize=output_size)

    # Get the predicted scores for each pixel
    _, pred = torch.max(scores, dim=1)
    pred = pred.cpu()[0].numpy()
    pred_tensor = torch.tensor(pred, dtype=torch.float32).unsqueeze(0).unsqueeze(0)  # Shape: (1, 1, height, width)
    pred_tensor = pred_tensor.cuda()

    # Forward pass through the autoencoder
    encoded_mask, reconstructed_mask = autoencoder(pred_tensor)

    # Calculate reconstruction loss
    reconstruction_loss = nn.MSELoss()
    loss = reconstruction_loss(reconstructed_mask, pred_tensor)

    return encoded_mask, loss.item()

# Example usage
normalization_params = {
    'mean': [0.485, 0.456, 0.406],
    'std': [0.229, 0.224, 0.225]
}

# Assuming `segmentation_module` and `simple_autoencoder` are defined and initialized
image_path = 'ADE_val_00001519.jpg'
encoded_mask, reconstruction_loss = process_image_and_get_loss(image_path, segmentation_module, simple_autoencoder, normalization_params)

print("Encoded Mask Shape:", encoded_mask.shape)
print("Reconstruction Loss:", reconstruction_loss)


(512, 768)
torch.Size([1, 1, 512, 768])
Encoded Mask Shape: torch.Size([1, 648])
Reconstruction Loss: 837.4664916992188


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load video from dropbox

In [ ]:
!wget -O dropbox_files.zip "https://www.dropbox.com/scl/fo/0q61ds2qxx6exwem6qbzc/AEMYOyGeSoJwJNIT1Y_mzlU?rlkey=krsdnjjby90kzo079szb8zy7g&st=1hucb4b9&dl=1"


--2024-11-20 17:58:41--  https://www.dropbox.com/scl/fo/0q61ds2qxx6exwem6qbzc/AEMYOyGeSoJwJNIT1Y_mzlU?rlkey=krsdnjjby90kzo079szb8zy7g&st=1hucb4b9&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc889812666912e9c14f0d4a3092.dl.dropboxusercontent.com/zip_download_get/CCLHm-0OESP6yLjKB4mi_PTgUCF_LjXZRmLjZwKyGNlvQLGDYBarrEtSLbzx4Nf43cKeZ-2KcEJhe2425bYvOAFJj0q46GrBBwaaYUwsHjAmyA# [following]
--2024-11-20 17:58:42--  https://uc889812666912e9c14f0d4a3092.dl.dropboxusercontent.com/zip_download_get/CCLHm-0OESP6yLjKB4mi_PTgUCF_LjXZRmLjZwKyGNlvQLGDYBarrEtSLbzx4Nf43cKeZ-2KcEJhe2425bYvOAFJj0q46GrBBwaaYUwsHjAmyA
Resolving uc889812666912e9c14f0d4a3092.dl.dropboxusercontent.com (uc889812666912e9c14f0d4a3092.dl.dropboxusercontent.com)... 162.125.1.15, 2620:100:6016:15::a27d:10f
Connecting to uc889812666912e9c14f0

In [ ]:
import zipfile

with zipfile.ZipFile("dropbox_files.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/dropbox_downloads")
print("Files extracted to /content/dropbox_downloads")


Files extracted to /content/dropbox_downloads


In [ ]:
import os
import shutil

# Define source directory and destination folder for .mp4 files
source_dir = "/content/dropbox_downloads/VirtualGuideDogLib"
mp4_output_dir = "/content/mp4_files"
os.makedirs(mp4_output_dir, exist_ok=True)

# Walk through the directory tree and copy all .mp4 files to the output directory
for root, dirs, files in os.walk(source_dir):
    print(root, dirs, files)
    for file in files:
        if file.endswith(".mp4"):
            full_path = os.path.join(root, file)
            shutil.copy(full_path, mp4_output_dir)
            print(f"Copied: {full_path}")

print(f"All .mp4 files are now in: {mp4_output_dir}")


/content/dropbox_downloads/VirtualGuideDogLib ['test'] []
/content/dropbox_downloads/VirtualGuideDogLib/test ['2022-04-04T12:36:21.745Z', '2022-04-04T12:43:51.211Z', '2022-04-04T13:01:31.083Z', '2022-04-04T13:15:19.655Z', '2022-04-04T12:14:43.949Z', '2022-04-04T12:51:41.445Z', '2022-04-04T12:56:37.203Z'] []
/content/dropbox_downloads/VirtualGuideDogLib/test/2022-04-04T12:36:21.745Z [] ['2022-04-04T12:36:21.745Z.csv', '2022-04-04T12:36:21.745Z.mp4']
Copied: /content/dropbox_downloads/VirtualGuideDogLib/test/2022-04-04T12:36:21.745Z/2022-04-04T12:36:21.745Z.mp4
/content/dropbox_downloads/VirtualGuideDogLib/test/2022-04-04T12:43:51.211Z [] ['2022-04-04T12:43:51.211Z.csv', '2022-04-04T12:43:51.211Z.mp4']
Copied: /content/dropbox_downloads/VirtualGuideDogLib/test/2022-04-04T12:43:51.211Z/2022-04-04T12:43:51.211Z.mp4
/content/dropbox_downloads/VirtualGuideDogLib/test/2022-04-04T13:01:31.083Z [] ['2022-04-04T13:01:31.083Z.mp4', '2022-04-04T13:01:31.083Z.csv']
Copied: /content/dropbox_download

In [ ]:
!wget -O dropbox_files2.zip "https://www.dropbox.com/scl/fo/d3663gx12my7sfzsmzbp4/APdg9wKx5wA7pvlMIu44Z-M?rlkey=iazskxeadgqrt5oqweb831ka3&e=1&st=p6sehk48&dl=1"

--2024-11-20 18:06:33--  https://www.dropbox.com/scl/fo/d3663gx12my7sfzsmzbp4/APdg9wKx5wA7pvlMIu44Z-M?rlkey=iazskxeadgqrt5oqweb831ka3&e=1&st=p6sehk48&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc9bccc537c2c6aed7f95f014ddc.dl.dropboxusercontent.com/zip_download_get/CCLmLQDUytZDZluWVExNwi3RnOAN3bXJXaROf4N3kSLMv0nBEHt8ryczOOkMGmMfUWAgbUlFopQIBX9Vhr5dChLRrvnUuPUbcZ7uqDj1iJR4uQ# [following]
--2024-11-20 18:06:35--  https://uc9bccc537c2c6aed7f95f014ddc.dl.dropboxusercontent.com/zip_download_get/CCLmLQDUytZDZluWVExNwi3RnOAN3bXJXaROf4N3kSLMv0nBEHt8ryczOOkMGmMfUWAgbUlFopQIBX9Vhr5dChLRrvnUuPUbcZ7uqDj1iJR4uQ
Resolving uc9bccc537c2c6aed7f95f014ddc.dl.dropboxusercontent.com (uc9bccc537c2c6aed7f95f014ddc.dl.dropboxusercontent.com)... 162.125.1.15, 2620:100:6016:15::a27d:10f
Connecting to uc9bccc537c2c6aed

In [ ]:
with zipfile.ZipFile("dropbox_files2.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/dropbox_downloads2")
print("Files extracted to /content/dropbox_downloads2")

Files extracted to /content/dropbox_downloads2


In [ ]:
ls /content/dropbox_downloads2

2022-04-04T18:41:15.478Z/  2022-04-04T19:22:45.749Z/  2022-09-07T17:48:53.625Z/
2022-04-04T18:47:26.422Z/  2022-04-04T19:23:24.989Z/  2022-09-07T17:49:31.370Z/
2022-04-04T18:48:45.704Z/  2022-04-04T19:24:10.208Z/  2022-09-07T17:50:02.923Z/
2022-04-04T18:58:12.589Z/  2022-04-04T19:28:21.899Z/  2022-09-07T17:50:31.458Z/
2022-04-04T18:59:05.875Z/  2022-04-04T19:28:40.596Z/  2022-09-07T17:51:24.478Z/
2022-04-04T19:01:16.917Z/  2022-04-04T19:28:57.167Z/  2022-09-07T18:15:13.682Z/
2022-04-04T19:02:28.038Z/  2022-09-07T17:42:05.175Z/  2022-09-07T18:15:32.232Z/
2022-04-04T19:04:56.768Z/  2022-09-07T17:43:08.884Z/  2022-09-07T18:15:43.550Z/
2022-04-04T19:06:29.642Z/  2022-09-07T17:43:16.918Z/  2022-09-07T18:16:01.302Z/
2022-04-04T19:07:55.583Z/  2022-09-07T17:43:39.620Z/  2022-09-07T18:21:23.281Z/
2022-04-04T19:09:43.644Z/  2022-09-07T17:44:43.991Z/  2022-09-07T18:21:40.015Z/
2022-04-04T19:10:28.196Z/  2022-09-07T17:45:09.377Z/  2022-09-07T18:23:12.931Z/
2022-04-04T19:16:45.412Z/  2022-09-07T17

In [ ]:
import os
import shutil

# Define source directory and destination folder for .mp4 files
source_dir = "/content/dropbox_downloads2/"
mp4_output_dir = "/content/mp4_files"
os.makedirs(mp4_output_dir, exist_ok=True)

# Walk through the directory tree and copy all .mp4 files to the output directory
for root, dirs, files in os.walk(source_dir):
    for file in files:
        if file.endswith(".mp4"):
            full_path = os.path.join(root, file)
            shutil.copy(full_path, mp4_output_dir)
            print(f"Copied: {full_path}")

print(f"All .mp4 files are now in: {mp4_output_dir}")


Copied: /content/dropbox_downloads2/2022-04-04T19:09:43.644Z/2022-04-04T19:09:43.644Z.mp4
Copied: /content/dropbox_downloads2/2022-04-04T19:28:21.899Z/2022-04-04T19:28:21.899Z.mp4
Copied: /content/dropbox_downloads2/2022-04-04T19:10:28.196Z/2022-04-04T19:10:28.196Z.mp4
Copied: /content/dropbox_downloads2/2022-04-04T19:17:50.959Z/2022-04-04T19:17:50.959Z.mp4
Copied: /content/dropbox_downloads2/2022-09-07T18:23:39.867Z/2022-09-07T18:23:39.867Z.mp4
Copied: /content/dropbox_downloads2/2022-09-07T18:25:51.421Z/2022-09-07T18:25:51.421Z.mp4
Copied: /content/dropbox_downloads2/2022-09-07T18:23:12.931Z/2022-09-07T18:23:12.931Z.mp4
Copied: /content/dropbox_downloads2/2022-09-07T17:51:24.478Z/2022-09-07T17:51:24.478Z.mp4
Copied: /content/dropbox_downloads2/2022-04-04T19:16:45.412Z/2022-04-04T19:16:45.412Z.mp4
Copied: /content/dropbox_downloads2/2022-04-04T19:04:56.768Z/2022-04-04T19:04:56.768Z.mp4
Copied: /content/dropbox_downloads2/2022-04-04T19:19:33.984Z/2022-04-04T19:19:33.984Z.mp4
Copied: /c

In [ ]:
!wget -O dropbox_files3.zip 'https://www.dropbox.com/scl/fo/6362bec5jklcio4jv0nyo/h?dl=1&rlkey=63a495mw7d5xw1k7kl4jwiea6&&dl=1'

--2024-11-20 18:11:51--  https://www.dropbox.com/scl/fo/6362bec5jklcio4jv0nyo/h?dl=1&rlkey=63a495mw7d5xw1k7kl4jwiea6&&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc3f23ee286c3b5ad28dc9d7dc3b.dl.dropboxusercontent.com/zip_download_get/CCI9aktG7E5-9QBBfCFIRJilVwBTmAaPu6AWTXWkxaqAFjoCkFLJJWTuYp6GvCNf_GHr-LJUplTWOVzier4RnjyZomIqORlwdY3uwlfSn_ahUw# [following]
--2024-11-20 18:11:53--  https://uc3f23ee286c3b5ad28dc9d7dc3b.dl.dropboxusercontent.com/zip_download_get/CCI9aktG7E5-9QBBfCFIRJilVwBTmAaPu6AWTXWkxaqAFjoCkFLJJWTuYp6GvCNf_GHr-LJUplTWOVzier4RnjyZomIqORlwdY3uwlfSn_ahUw
Resolving uc3f23ee286c3b5ad28dc9d7dc3b.dl.dropboxusercontent.com (uc3f23ee286c3b5ad28dc9d7dc3b.dl.dropboxusercontent.com)... 162.125.1.15, 2620:100:6016:15::a27d:10f
Connecting to uc3f23ee286c3b5ad28dc9d7dc3b.dl.dropboxuserconten

In [ ]:
with zipfile.ZipFile("dropbox_files3.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/dropbox_downloads3")
print("Files extracted to /content/dropbox_downloads3")

Files extracted to /content/dropbox_downloads3


In [ ]:
source_dir = "/content/dropbox_downloads3/"
mp4_output_dir = "/content/mp4_files"
os.makedirs(mp4_output_dir, exist_ok=True)

# Walk through the directory tree and copy all .mp4 files to the output directory
for root, dirs, files in os.walk(source_dir):
    for file in files:
        if file.endswith(".mp4"):
            full_path = os.path.join(root, file)
            shutil.copy(full_path, mp4_output_dir)
            print(f"Copied: {full_path}")

print(f"All .mp4 files are now in: {mp4_output_dir}")

Copied: /content/dropbox_downloads3/2022-04-04T18:41:15.478Z/2022-04-04T18:41:15.478Z.mp4
Copied: /content/dropbox_downloads3/2022-04-04T19:28:57.167Z/2022-04-04T19:28:57.167Z.mp4
Copied: /content/dropbox_downloads3/2022-04-04T19:07:55.583Z/2022-04-04T19:07:55.583Z.mp4
Copied: /content/dropbox_downloads3/2022-04-04T18:58:12.589Z/2022-04-04T18:58:12.589Z.mp4
Copied: /content/dropbox_downloads3/2022-04-04T18:59:05.875Z/2022-04-04T18:59:05.875Z.mp4
Copied: /content/dropbox_downloads3/2022-04-04T19:21:41.445Z/2022-04-04T19:21:41.445Z.mp4
All .mp4 files are now in: /content/mp4_files


In [ ]:
!ls '/content/drive/My Drive/Capstone - AI Guide Dog 2024'

 11632_standup_quadchart_Nov21.pptx		       'Guide Dog 2024 Master.gdoc'
 11632_standup_quadchart_Nov7.pptx		        Meeting_minutes.gdoc
 11632_standup_quadchart_Oct10.pptx		       'meeting recording'
 11632_standup_quadchart_Sept26.pptx		        output_frames
 11634_ice_breaker_AI_Guide_Dog.gdoc		        Rough_work.gdoc
'2022 docs'					        segmentation_masks
'AI_Guide_Dog_2024_-_Capstone_Proposal_(2).pdf'         video1098796292.mp4
'AI Guide Dog 2024 Midterm Presentation.gslides'        video1194195383.mp4
'AI Guide Dog 2024 Spring Final Presentation.gslides'   video1753052460.mp4
'AI Guide Dog Fall Plan 2024.pptx'		        video2900402067.mp4
 AI_Guide_Dog_MidTerm_Presentation.pptx		        videos
'Baseline Dataset'				       'Vision Document'


In [ ]:
source_dir = '/content/mp4_files'  # Update with your source directory path
destination_dir = '/content/drive/My Drive/Capstone - AI Guide Dog 2024/data_videos'  # Update with your target directory in Google Drive

# Copy directory to Google Drive
shutil.copytree(source_dir, destination_dir)

print(f"Directory copied to: {destination_dir}")

Directory copied to: /content/drive/My Drive/Capstone - AI Guide Dog 2024/data_videos


# get all frames

In [ ]:
import os
import subprocess

# Directory containing all .mp4 files
mp4_dir = "/content/mp4_files"
# Directory to save extracted frames
frames_dir = "/content/extracted_frames"
os.makedirs(frames_dir, exist_ok=True)

# Loop through all .mp4 files
for mp4_file in os.listdir(mp4_dir):
    if mp4_file.endswith(".mp4"):
        video_path = os.path.join(mp4_dir, mp4_file)
        video_name = os.path.splitext(mp4_file)[0]  # Get video name without extension

        # ffmpeg command to extract frames with unique filenames
        cmd = [
            "ffmpeg", "-i", video_path,
            "-vf", "fps=2,scale=480:270",
            f"{frames_dir}/{video_name}_frame_%04d.png"
        ]

        # Run the command
        print(f"Extracting frames for: {mp4_file}")
        subprocess.run(cmd)

print("Frame extraction complete!")


Extracting frames for: 2022-04-04T18:48:45.704Z.mp4
Extracting frames for: 2022-04-04T19:28:40.596Z.mp4
Extracting frames for: 2022-04-04T12:56:37.203Z.mp4
Extracting frames for: 2022-04-04T19:22:45.749Z.mp4
Extracting frames for: 2022-04-04T16:14:28.350Z.mp4
Extracting frames for: 2022-09-07T17:48:18.039Z.mp4
Extracting frames for: 2022-09-07T18:23:12.931Z.mp4
Extracting frames for: 2022-09-07T17:50:31.458Z.mp4
Extracting frames for: 2022-04-04T19:28:57.167Z.mp4
Extracting frames for: 2022-04-04T12:51:41.445Z.mp4
Extracting frames for: 2022-09-07T17:47:02.517Z.mp4
Extracting frames for: 2022-04-04T18:59:05.875Z.mp4
Extracting frames for: 2022-04-04T16:18:27.292Z.mp4
Extracting frames for: 2022-04-04T19:01:16.917Z.mp4
Extracting frames for: 2022-04-04T13:15:19.655Z.mp4
Extracting frames for: 2022-04-04T19:09:43.644Z.mp4
Extracting frames for: 2022-04-04T19:24:10.208Z.mp4
Extracting frames for: 2022-04-04T19:28:21.899Z.mp4
Extracting frames for: 2022-09-07T18:15:13.682Z.mp4
Extracting f

In [ ]:
source_dir = '/content/extracted_frames'  # Update with your source directory path
destination_dir = '/content/drive/My Drive/Capstone - AI Guide Dog 2024/extracted_frames'  # Update with your target directory in Google Drive

# Copy directory to Google Drive
shutil.copytree(source_dir, destination_dir)

print(f"Directory copied to: {destination_dir}")

Directory copied to: /content/drive/My Drive/Capstone - AI Guide Dog 2024/extracted_frames


# Process image and store segments

In [ ]:
import torch
import torch.nn as nn
import torchvision
import PIL.Image
import numpy as np
from torch.utils.data import Dataset, DataLoader
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import random
def extract_and_save_segmentations(image_dir, segmentation_module, output_dir, batch_size=4):
    """
    Extract segmentations for all images in a directory and save them.

    Args:
        image_dir: Directory containing input images
        segmentation_module: Pretrained segmentation model
        output_dir: Directory to save segmentation masks
        batch_size: Batch size for processing
    """
    os.makedirs(output_dir, exist_ok=True)

    # Image transformation pipeline
    transform = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])

    # Get list of all images
    image_files = [f for f in os.listdir(image_dir)]

    for img_file in tqdm(image_files, desc="Processing images"):
        try:
            # Load and process image
            img_path = os.path.join(image_dir, img_file)
            pil_image = PIL.Image.open(img_path).convert('RGB')
            img_original = numpy.array(pil_image)
            img_data = pil_to_tensor(pil_image)
            singleton_batch = {'img_data': img_data[None].cuda()}
            output_size = img_data.shape[1:]

            # Generate segmentation
            with torch.no_grad():
                scores = segmentation_module(singleton_batch, segSize=output_size)

            # Get prediction
            _, pred = torch.max(scores, dim=1)
            pred = pred.cpu()[0].numpy()
            pred_tensor = torch.tensor(pred, dtype=torch.float32).unsqueeze(0).unsqueeze(0)

            # Save segmentation mask
            output_path = os.path.join(output_dir, f"{os.path.splitext(img_file)[0]}_segment.npy")
            np.save(output_path, pred)

        except Exception as e:
            print(f"Error processing {img_file}: {str(e)}")
            continue

In [ ]:
image_dir = 'extracted_frames'
segmentation_output_dir = 'extracted_masks'

# Load and normalize one image as a singleton tensor batch
pil_to_tensor = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(
        mean=[0.485, 0.456, 0.406], # These are RGB mean+std values
        std=[0.229, 0.224, 0.225])  # across a large photo dataset.
])


In [ ]:
#### NO RERUN #######
extract_and_save_segmentations(
    image_dir=image_dir,
    segmentation_module=segmentation_module,  # Your pretrained segmentation module
    output_dir=segmentation_output_dir
)

Processing images: 100%|██████████| 7007/7007 [07:00<00:00, 16.66it/s]


In [ ]:
source_dir = '/content/extracted_masks'  # Update with your source directory path
destination_dir = '/content/drive/My Drive/Capstone - AI Guide Dog 2024/extracted_masks'  # Update with your target directory in Google Drive

# Copy directory to Google Drive
shutil.copytree(source_dir, destination_dir)

print(f"Directory copied to: {destination_dir}")

Directory copied to: /content/drive/My Drive/Capstone - AI Guide Dog 2024/extracted_masks


In [ ]:


def train_autoencoder(autoencoder, segmentation_dir, num_epochs=100, learning_rate=0.001):
    """
    Train autoencoder on stored segmentation masks, processing one image at a time.

    Args:
        autoencoder: Autoencoder model
        segmentation_dir: Directory containing stored segmentation masks
        num_epochs: Number of training epochs
        learning_rate: Learning rate for optimizer
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    autoencoder = autoencoder.to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(autoencoder.parameters(), lr=learning_rate)

    # Get list of all segmentation files
    seg_files = [f for f in os.listdir(segmentation_dir)][:100]
    print(seg_files)

    # Training loop
    for epoch in range(num_epochs):
        total_loss = 0
        # Shuffle files at the start of each epoch
        random.shuffle(seg_files)

        for seg_file in tqdm(seg_files, desc=f"Epoch {epoch+1}/{num_epochs}"):
            try:
                # Load single segmentation mask
                seg_path = os.path.join(segmentation_dir, seg_file)
                seg_mask = np.load(seg_path)
                seg_tensor = torch.tensor(seg_mask, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
                seg_tensor = seg_tensor.to(device)

                # Forward pass
                encoded, reconstructed = autoencoder(seg_tensor)

                # Calculate loss
                loss = criterion(reconstructed, seg_tensor)
                print(loss)

                # Backward pass
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                total_loss += loss.item()

            except Exception as e:
                print(f"Error processing {seg_file}: {str(e)}")
                continue

        avg_loss = total_loss / len(seg_files)
        print(f"Epoch {epoch+1}, Average Loss: {avg_loss:.6f}")

        # Save checkpoint every 10 epochs
        if (epoch + 1) % 10 == 0:
            checkpoint_path = f'autoencoder_checkpoint_epoch_{epoch+1}.pth'
            torch.save({
                'epoch': epoch,
                'model_state_dict': autoencoder.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': avg_loss,
            }, checkpoint_path)

    return autoencoder

In [ ]:
k = 648  # Encoded dimension size
autoencoder = MaskAutoencoder(height=270, width=480, k=k)


In [ ]:
trained_autoencoder = train_autoencoder(
        autoencoder=autoencoder,
        segmentation_dir=segmentation_output_dir,
        num_epochs=1
    )

['frame_0001_seg.npy', 'frame_0002_seg.npy', 'frame_0003_seg.npy', 'frame_0004_seg.npy', 'frame_0005_seg.npy', 'frame_0006_seg.npy', 'frame_0007_seg.npy', 'frame_0008_seg.npy', 'frame_0009_seg.npy', 'frame_0010_seg.npy', 'frame_0011_seg.npy', 'frame_0012_seg.npy', 'frame_0013_seg.npy', 'frame_0014_seg.npy', 'frame_0015_seg.npy', 'frame_0016_seg.npy', 'frame_0017_seg.npy', 'frame_0018_seg.npy', 'frame_0019_seg.npy', 'frame_0020_seg.npy', 'frame_0021_seg.npy', 'frame_0022_seg.npy', 'frame_0023_seg.npy', 'frame_0024_seg.npy', 'frame_0025_seg.npy', 'frame_0026_seg.npy', 'frame_0027_seg.npy', 'frame_0028_seg.npy', 'frame_0029_seg.npy', 'frame_0030_seg.npy', 'frame_0031_seg.npy', 'frame_0032_seg.npy', 'frame_0033_seg.npy', 'frame_0034_seg.npy', 'frame_0035_seg.npy', 'frame_0036_seg.npy', 'frame_0037_seg.npy', 'frame_0038_seg.npy', 'frame_0039_seg.npy', 'frame_0040_seg.npy', 'frame_0041_seg.npy', 'frame_0042_seg.npy', 'frame_0043_seg.npy', 'frame_0044_seg.npy', 'frame_0045_seg.npy', 'frame_00

Epoch 1/1:   1%|          | 1/100 [00:00<01:06,  1.49it/s]

tensor(96.4983, device='cuda:0', grad_fn=<MseLossBackward0>)
Error processing frame_0038_seg.npy: CUDA out of memory. Tried to allocate 1.26 GiB. GPU 0 has a total capacity of 14.75 GiB of which 537.06 MiB is free. Process 22363 has 14.22 GiB memory in use. Of the allocated memory 12.83 GiB is allocated by PyTorch, and 1.26 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)


Epoch 1/1:   2%|▏         | 2/100 [00:01<00:53,  1.82it/s]

tensor(837.6417, device='cuda:0', grad_fn=<MseLossBackward0>)
Error processing frame_0004_seg.npy: 'exp_avg'


...

tensor(629.7054, device='cuda:0', grad_fn=<MseLossBackward0>)
Error processing frame_0005_seg.npy: 'exp_avg'
tensor(392.1633, device='cuda:0', grad_fn=<MseLossBackward0>)
Error processing frame_0018_seg.npy: 'exp_avg'
tensor(862.4950, device='cuda:0', grad_fn=<MseLossBackward0>)
Error processing frame_0070_seg.npy: 'exp_avg'
tensor(757.5562, device='cuda:0', grad_fn=<MseLossBackward0>)
Error processing frame_0013_seg.npy: 'exp_avg'
tensor(930.7218, device='cuda:0', grad_fn=<MseLossBackward0>)
Error processing frame_0016_seg.npy: 'exp_avg'
tensor(739.2895, device='cuda:0', grad_fn=<MseLossBackward0>)
Error processing frame_0086_seg.npy: 'exp_avg'


Epoch 1/1:  94%|█████████▍| 94/100 [00:07<00:00, 22.85it/s]

tensor(584.7433, device='cuda:0', grad_fn=<MseLossBackward0>)
Error processing frame_0031_seg.npy: 'exp_avg'
tensor(316.5999, device='cuda:0', grad_fn=<MseLossBackward0>)
Error processing frame_0027_seg.npy: 'exp_avg'
tensor(349.2674, device='cuda:0', grad_fn=<MseLossBackward0>)
Error processing frame_0033_seg.npy: 'exp_avg'
tensor(439.7694, device='cuda:0', grad_fn=<MseLossBackward0>)
Error processing frame_0076_seg.npy: 'exp_avg'
tensor(272.4155, device='cuda:0', grad_fn=<MseLossBackward0>)
Error processing frame_0090_seg.npy: 'exp_avg'
tensor(236.0028, device='cuda:0', grad_fn=<MseLossBackward0>)
Error processing frame_0092_seg.npy: 'exp_avg'


Epoch 1/1: 100%|██████████| 100/100 [00:08<00:00, 12.26it/s]

tensor(996.3928, device='cuda:0', grad_fn=<MseLossBackward0>)
Error processing frame_0010_seg.npy: 'exp_avg'
tensor(321.0468, device='cuda:0', grad_fn=<MseLossBackward0>)
Error processing frame_0083_seg.npy: 'exp_avg'
tensor(322.7357, device='cuda:0', grad_fn=<MseLossBackward0>)
Error processing frame_0060_seg.npy: 'exp_avg'
tensor(604.5424, device='cuda:0', grad_fn=<MseLossBackward0>)
Error processing frame_0034_seg.npy: 'exp_avg'
tensor(640.1530, device='cuda:0', grad_fn=<MseLossBackward0>)
Error processing frame_0028_seg.npy: 'exp_avg'
Epoch 1, Average Loss: 0.000000


In [ ]:
torch.save(trained_autoencoder.state_dict(), 'trained_autoencoder_final.pth')

In [ ]:
# with dataloader for efficiency
def train_autoencoder(autoencoder, segmentation_dir, num_epochs=100, learning_rate=0.001, batch_size=16):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    autoencoder = autoencoder.to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(autoencoder.parameters(), lr=learning_rate)

    # Create a DataLoader with batching


    for epoch in range(num_epochs):
        total_loss = 0
        for batch in tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            seg_tensor = batch.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            encoded, reconstructed = autoencoder(seg_tensor)

            # Compute loss
            loss = criterion(reconstructed, seg_tensor)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(dataloader)
        print(f"Epoch {epoch+1}, Average Loss: {avg_loss:.6f}")

        if (epoch + 1) % 10 == 0:
            checkpoint_path = f'autoencoder_checkpoint_epoch_{epoch+1}.pth'
            torch.save({
                'epoch': epoch,
                'model_state_dict': autoencoder.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': avg_loss,
            }, checkpoint_path)
    return autoencoder

class SegmentationDataset(Dataset):
    def __init__(self, segmentation_dir, file_list):
        self.segmentation_dir = segmentation_dir
        self.file_list = file_list

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        seg_file = self.file_list[idx]
        seg_path = os.path.join(self.segmentation_dir, seg_file)
        seg_mask = np.load(seg_path)
        return torch.tensor(seg_mask, dtype=torch.float32).unsqueeze(0)

In [ ]:
k = 648  # Encoded dimension size
autoencoder = MaskAutoencoder(height=270, width=480, k=k)


In [ ]:
# with train-test split
def train_autoencoder(autoencoder, segmentation_dir, num_epochs=100, learning_rate=0.001, batch_size=16, eval_frequency=10):
    # Create dataset
    seg_files = [f for f in os.listdir(segmentation_dir)]
    full_dataset = SegmentationDataset(segmentation_dir, seg_files)

    # Split dataset
    train_size = int(0.8 * len(full_dataset))
    test_size = len(full_dataset) - train_size
    train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

    # Create DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    autoencoder = autoencoder.to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(autoencoder.parameters(), lr=learning_rate)

    # Training loop with test evaluation
    for epoch in range(num_epochs):
        # Training phase
        autoencoder.train()
        train_loss = 0
        for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
            seg_tensor = batch.to(device)

            optimizer.zero_grad()
            encoded, reconstructed = autoencoder(seg_tensor)
            loss = criterion(reconstructed, seg_tensor)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        # Print epoch statistics
        print(f"Epoch {epoch+1}")
        print(f"Train Loss: {train_loss/len(train_loader):.6f}")

        # Validation phase
        if (epoch + 1) % eval_frequency == 0:
          autoencoder.eval()
          val_loss = 0
          with torch.no_grad():
              for batch in test_loader:
                  seg_tensor = batch.to(device)
                  encoded, reconstructed = autoencoder(seg_tensor)
                  loss = criterion(reconstructed, seg_tensor)
                  val_loss += loss.item()
              print(f"Validation Loss: {val_loss/len(test_loader):.6f}")

        if (epoch + 1) % 10 == 0:
            checkpoint_path = f'autoencoder_checkpoint_epoch_{epoch+1}.pth'
            torch.save({
                'epoch': epoch,
                'model_state_dict': autoencoder.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': avg_loss,
            }, checkpoint_path)

    return autoencoder

In [ ]:
trained_autoencoder = train_autoencoder(
        autoencoder=autoencoder,
        segmentation_dir='extracted_masks',
        num_epochs=50
    )

Epoch 1/50: 100%|██████████| 438/438 [02:39<00:00,  2.74it/s]


Epoch 1, Average Loss: 653.636256


Epoch 2/50: 100%|██████████| 438/438 [02:31<00:00,  2.88it/s]


Epoch 2, Average Loss: 653.623959


Epoch 3/50: 100%|██████████| 438/438 [02:26<00:00,  3.00it/s]


...

Epoch 48, Average Loss: 653.522894


Epoch 49/50: 100%|██████████| 438/438 [02:05<00:00,  3.49it/s]


Epoch 49, Average Loss: 653.558751


Epoch 50/50: 100%|██████████| 438/438 [02:05<00:00,  3.49it/s]

Epoch 50, Average Loss: 653.513968


In [ ]:
source_dir = 'content'  # Update with your directory path
zip_file = '/content/directory_name.zip'  # Update with your desired zip file name

# Compress the directory into a zip file
shutil.make_archive(zip_file.replace('.zip', ''), 'zip', source_dir)

print(f"Directory compressed to {zip_file}")

In [ ]:
# Epoch 1/50: 100%|██████████| 438/438 [02:39<00:00,  2.74it/s]
# Epoch 1, Average Loss: 653.636256
# Epoch 2/50: 100%|██████████| 438/438 [02:31<00:00,  2.88it/s]
# Epoch 2, Average Loss: 653.623959
# Epoch 3/50: 100%|██████████| 438/438 [02:26<00:00,  3.00it/s]
# Epoch 3, Average Loss: 653.546840
# Epoch 4/50: 100%|██████████| 438/438 [02:25<00:00,  3.02it/s]
# Epoch 4, Average Loss: 653.611095
# Epoch 5/50: 100%|██████████| 438/438 [02:23<00:00,  3.04it/s]
# Epoch 5, Average Loss: 653.597986
# Epoch 6/50: 100%|██████████| 438/438 [02:21<00:00,  3.10it/s]
# Epoch 6, Average Loss: 653.536078
# Epoch 7/50: 100%|██████████| 438/438 [02:20<00:00,  3.12it/s]
# Epoch 7, Average Loss: 653.518660
# Epoch 8/50: 100%|██████████| 438/438 [02:16<00:00,  3.20it/s]
# Epoch 8, Average Loss: 653.565742
# Epoch 9/50: 100%|██████████| 438/438 [02:14<00:00,  3.26it/s]
# Epoch 9, Average Loss: 653.552168
# Epoch 10/50: 100%|██████████| 438/438 [02:10<00:00,  3.36it/s]
# Epoch 10, Average Loss: 653.536049
# Epoch 11/50: 100%|██████████| 438/438 [02:05<00:00,  3.49it/s]
# Epoch 11, Average Loss: 653.536043
# Epoch 12/50: 100%|██████████| 438/438 [02:05<00:00,  3.49it/s]
# Epoch 12, Average Loss: 653.529989
# Epoch 13/50: 100%|██████████| 438/438 [02:05<00:00,  3.49it/s]
# Epoch 13, Average Loss: 653.529931
# Epoch 14/50: 100%|██████████| 438/438 [02:05<00:00,  3.49it/s]
# Epoch 14, Average Loss: 653.574474
# Epoch 15/50: 100%|██████████| 438/438 [02:05<00:00,  3.49it/s]
# Epoch 15, Average Loss: 653.567729
# Epoch 16/50: 100%|██████████| 438/438 [02:05<00:00,  3.50it/s]
# Epoch 16, Average Loss: 653.557613
# Epoch 17/50: 100%|██████████| 438/438 [02:05<00:00,  3.49it/s]
# Epoch 17, Average Loss: 653.540401
# Epoch 18/50: 100%|██████████| 438/438 [02:05<00:00,  3.50it/s]
# Epoch 18, Average Loss: 653.526081
# Epoch 19/50: 100%|██████████| 438/438 [02:05<00:00,  3.49it/s]
# Epoch 19, Average Loss: 653.520010
# Epoch 20/50: 100%|██████████| 438/438 [02:05<00:00,  3.49it/s]
# Epoch 20, Average Loss: 653.569354
# Epoch 21/50: 100%|██████████| 438/438 [02:05<00:00,  3.48it/s]
# Epoch 21, Average Loss: 653.597220
# Epoch 22/50: 100%|██████████| 438/438 [02:05<00:00,  3.49it/s]
# Epoch 22, Average Loss: 653.620864
# Epoch 23/50: 100%|██████████| 438/438 [02:05<00:00,  3.49it/s]
# Epoch 23, Average Loss: 653.535454
# Epoch 24/50: 100%|██████████| 438/438 [02:05<00:00,  3.49it/s]
# Epoch 24, Average Loss: 653.533524
# Epoch 25/50: 100%|██████████| 438/438 [02:05<00:00,  3.49it/s]
# Epoch 25, Average Loss: 653.540995
# Epoch 26/50: 100%|██████████| 438/438 [02:05<00:00,  3.49it/s]
# Epoch 26, Average Loss: 653.543492
# Epoch 27/50: 100%|██████████| 438/438 [02:05<00:00,  3.49it/s]
# Epoch 27, Average Loss: 653.531279
# Epoch 28/50: 100%|██████████| 438/438 [02:05<00:00,  3.48it/s]
# Epoch 28, Average Loss: 653.555228
# Epoch 29/50: 100%|██████████| 438/438 [02:05<00:00,  3.49it/s]
# Epoch 29, Average Loss: 653.507739
# Epoch 30/50: 100%|██████████| 438/438 [02:05<00:00,  3.49it/s]
# Epoch 30, Average Loss: 653.511117
# Epoch 31/50: 100%|██████████| 438/438 [02:05<00:00,  3.48it/s]
# Epoch 31, Average Loss: 653.585588
# Epoch 32/50: 100%|██████████| 438/438 [02:05<00:00,  3.48it/s]
# Epoch 32, Average Loss: 653.497608
# Epoch 33/50: 100%|██████████| 438/438 [02:05<00:00,  3.49it/s]
# Epoch 33, Average Loss: 653.513476
# Epoch 34/50: 100%|██████████| 438/438 [02:05<00:00,  3.49it/s]
# Epoch 34, Average Loss: 653.532347
# Epoch 35/50: 100%|██████████| 438/438 [02:05<00:00,  3.48it/s]
# Epoch 35, Average Loss: 653.567354
# Epoch 36/50: 100%|██████████| 438/438 [02:05<00:00,  3.49it/s]
# Epoch 36, Average Loss: 653.548174
# Epoch 37/50: 100%|██████████| 438/438 [02:05<00:00,  3.48it/s]
# Epoch 37, Average Loss: 653.590653
# Epoch 38/50: 100%|██████████| 438/438 [02:05<00:00,  3.49it/s]
# Epoch 38, Average Loss: 653.529242
# Epoch 39/50: 100%|██████████| 438/438 [02:05<00:00,  3.48it/s]
# Epoch 39, Average Loss: 653.551089
# Epoch 40/50: 100%|██████████| 438/438 [02:05<00:00,  3.49it/s]
# Epoch 40, Average Loss: 653.519890
# Epoch 41/50: 100%|██████████| 438/438 [02:05<00:00,  3.48it/s]
# Epoch 41, Average Loss: 653.542692
# Epoch 42/50: 100%|██████████| 438/438 [02:05<00:00,  3.49it/s]
# Epoch 42, Average Loss: 653.502513
# Epoch 43/50: 100%|██████████| 438/438 [02:05<00:00,  3.48it/s]
# Epoch 43, Average Loss: 653.528728
# Epoch 44/50: 100%|██████████| 438/438 [02:05<00:00,  3.49it/s]
# Epoch 44, Average Loss: 653.546500
# Epoch 45/50: 100%|██████████| 438/438 [02:05<00:00,  3.49it/s]
# Epoch 45, Average Loss: 653.498348
# Epoch 46/50: 100%|██████████| 438/438 [02:05<00:00,  3.48it/s]
# Epoch 46, Average Loss: 653.560366
# Epoch 47/50: 100%|██████████| 438/438 [02:05<00:00,  3.49it/s]
# Epoch 47, Average Loss: 653.558996
# Epoch 48/50: 100%|██████████| 438/438 [02:05<00:00,  3.49it/s]
# Epoch 48, Average Loss: 653.522894
# Epoch 49/50: 100%|██████████| 438/438 [02:05<00:00,  3.49it/s]
# Epoch 49, Average Loss: 653.558751
# Epoch 50/50: 100%|██████████| 438/438 [02:05<00:00,  3.49it/s]Epoch 50, Average Loss: 653.513968